In [2]:
import Pkg; Pkg.add("OffsetArrays")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %60.6 % Resolving package versions...
 Installed OffsetArrays ─ v0.11.2
  Updating `~/.julia/environments/v1.3/Project.toml`
  [6fe1bfb0] + OffsetArrays v0.11.2
  Updating `~/.julia/environments/v1.3/Manifest.toml`
  [6fe1bfb0] + OffsetArrays v0.11.2


In [43]:
#using someone's julia code for day 5

using OffsetArrays

struct instruction
    opcode::Int
    mode::Array{Int}
end

function instruction(x::Int)
    d = digits(x,pad=5)
    instruction(d[1]+10*d[2],d[3:end])
end

# Not in base yet - Just to make input nice
function input(prompt::String="")::String
    print(prompt)
    return chomp(readline())
end

function value(program::OffsetArray,input,mode) 
    if mode == 1 # Immediate
        return input 
    elseif mode == 0 # Position
        return program[input] 
    end
end

function values(program,inst,i)
    # This will determine how many parameters to take
    if inst.opcode in [1,2,5,6,7,8]
        a = value(program,program[i+1],inst.mode[1])
        b = value(program,program[i+2],inst.mode[2])
        return a,b
    elseif inst.opcode in [4]
        return value(program,program[i+1],inst.mode[1])
    end
end


function runIntcode!(program::OffsetArray,input_values)
    i = 0
    last_output=0
    input_track = 1
    while i < length(program)
        # First is an instruction
        inst = instruction(program[i])
        if inst.opcode == 1
            # add
            program[program[i+3]] = +(values(program,inst,i)...)
            i += 4
        elseif inst.opcode == 2
            # mul
            program[program[i+3]] = *(values(program,inst,i)...)
            i += 4
        elseif inst.opcode == 3
            # read
            program[program[i+1]] = input_values[input_track] #parse(Int,input("Input: "))
            input_track+=1
            i += 2
        elseif inst.opcode == 4
            # write
            println("Output: $(values(program,inst,i))")
            last_output = values(program,inst,i)
            i += 2
        elseif inst.opcode == 5
            # jmp-if-true
            a,b = values(program,inst,i)
            a != 0 ? i = b : i += 3
        elseif inst.opcode == 6
            # jmp-if-false
            a,b = values(program,inst,i)
            a == 0 ? i = b : i += 3
        elseif inst.opcode == 7
            # less
            program[program[i+3]] = Int(<(values(program,inst,i)...))
            i += 4
        elseif inst.opcode == 8
            # eql
            program[program[i+3]] = Int(==(values(program,inst,i)...))
            i += 4
        elseif inst.opcode == 99
            # exit
            break
        else
            print("Invalid Intcode")
            break
        end
    end
    println("Program Complete")
    return last_output
end

program = OffsetArray([parse(Int64,i) for i in split(readline("input-5.txt"),',')],-1)

runIntcode!(program,1)


Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 0
Output: 9219874
Program Complete


9219874

In [45]:
og_program = OffsetArray([parse(Int64,i) for i in split("3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0",',')],-1)

output=0

for phase in [4,3,2,1,0]
    og_program = OffsetArray([parse(Int64,i) for i in split("3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0",',')],-1)
    output = runIntcode!(og_program,[phase,output])
end

@assert output == 43210

Output: 4
Program Complete
Output: 43
Program Complete
Output: 432
Program Complete
Output: 4321
Program Complete
Output: 43210
Program Complete


In [47]:
output=0

for phase in [0,1,2,3,4]
    og_program = OffsetArray([parse(Int64,i) for i in split("3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0",',')],-1)
    output = runIntcode!(og_program,[phase,output])
end

@assert output == 54321

Output: 5
Program Complete
Output: 54
Program Complete
Output: 543
Program Complete
Output: 5432
Program Complete
Output: 54321
Program Complete


In [18]:
for i in [4,3,2,1,0]
    println(i)
end

4
3
2
1
0


In [48]:
import Pkg; Pkg.add("Combinatorics") 

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
 Installed RecipesBase ─── v0.7.0
 Installed Combinatorics ─ v1.0.0
 Installed Polynomials ─── v0.6.0
  Updating `~/.julia/environments/v1.3/Project.toml`
  [861a8166] + Combinatorics v1.0.0
  Updating `~/.julia/environments/v1.3/Manifest.toml`
  [861a8166] + Combinatorics v1.0.0
  [f27b6e38] + Polynomials v0.6.0
  [3cdcf5f2] + RecipesBase v0.7.0


In [53]:
using Combinatorics
x = permutations([0,1,2,3,4]) |> collect

120-element Array{Array{Int64,1},1}:
 [0, 1, 2, 3, 4]
 [0, 1, 2, 4, 3]
 [0, 1, 3, 2, 4]
 [0, 1, 3, 4, 2]
 [0, 1, 4, 2, 3]
 [0, 1, 4, 3, 2]
 [0, 2, 1, 3, 4]
 [0, 2, 1, 4, 3]
 [0, 2, 3, 1, 4]
 [0, 2, 3, 4, 1]
 [0, 2, 4, 1, 3]
 [0, 2, 4, 3, 1]
 [0, 3, 1, 2, 4]
 ⋮              
 [4, 2, 0, 1, 3]
 [4, 2, 0, 3, 1]
 [4, 2, 1, 0, 3]
 [4, 2, 1, 3, 0]
 [4, 2, 3, 0, 1]
 [4, 2, 3, 1, 0]
 [4, 3, 0, 1, 2]
 [4, 3, 0, 2, 1]
 [4, 3, 1, 0, 2]
 [4, 3, 1, 2, 0]
 [4, 3, 2, 0, 1]
 [4, 3, 2, 1, 0]

In [80]:
biggest = 0
code_input = "3,8,1001,8,10,8,105,1,0,0,21,42,67,84,109,126,207,288,369,450,99999,3,9,102,4,9,9,1001,9,4,9,102,2,9,9,101,2,9,9,4,9,99,3,9,1001,9,5,9,1002,9,5,9,1001,9,5,9,1002,9,5,9,101,5,9,9,4,9,99,3,9,101,5,9,9,1002,9,3,9,1001,9,2,9,4,9,99,3,9,1001,9,2,9,102,4,9,9,101,2,9,9,102,4,9,9,1001,9,2,9,4,9,99,3,9,102,2,9,9,101,5,9,9,1002,9,2,9,4,9,99,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,1,9,4,9,99,3,9,1001,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1002,9,2,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,101,1,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,101,2,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,99,3,9,1001,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,101,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1001,9,2,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,102,2,9,9,4,9,3,9,101,1,9,9,4,9,99,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,101,2,9,9,4,9,3,9,1002,9,2,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,1,9,4,9,3,9,1002,9,2,9,4,9,3,9,101,1,9,9,4,9,3,9,102,2,9,9,4,9,3,9,1001,9,2,9,4,9,99"

for seq in x
    
    output=0

    for phase in seq
        og_program = OffsetArray([parse(Int64,i) for i in split(code_input,',')],-1)
        output = runIntcode!(og_program,[phase,output])
    end
    #print("myoutput",typeof(output))
    if output>biggest
        #print("got new biggest: ",biggest," ",output)
        biggest=output
        bestseq=seq

    end
end

    


Output: 10
Program Complete
Output: 405
Program Complete
Output: 1232
Program Complete
Output: 19754
Program Complete
Output: 79026
Program Complete
Output: 10
Program Complete
Output: 405
Program Complete
Output: 1232
Program Complete
Output: 4938
Program Complete
Output: 79050
Program Complete
Output: 10
Program Complete
Output: 405
Program Complete
Output: 6522
Program Complete
Output: 19583
Program Complete
Output: 78342
Program Complete
Output: 10
Program Complete
Output: 405
Program Complete
Output: 6522
Program Complete
Output: 26098
Program Complete
Output: 78311
Program Complete
Output: 10
Program Complete
Output: 405
Program Complete
Output: 1630
Program Complete
Output: 4907
Program Complete
Output: 78554
Program Complete
Output: 10
Program Complete
Output: 405
Program Complete
Output: 1630
Program Complete
Output: 26122
Program Complete
Output: 78383
Program Complete
Output: 10
Program Complete
Output: 47
Program Complete
Output: 1330
Program Complete
Output: 21322
Program 

Program Complete
Output: 16850
Program Complete
Output: 269642
Program Complete
Output: 17
Program Complete
Output: 78
Program Complete
Output: 2105
Program Complete
Output: 33722
Program Complete
Output: 269786
Program Complete
Output: 17
Program Complete
Output: 78
Program Complete
Output: 1290
Program Complete
Output: 10330
Program Complete
Output: 258405
Program Complete
Output: 17
Program Complete
Output: 78
Program Complete
Output: 1290
Program Complete
Output: 32405
Program Complete
Output: 259250
Program Complete
Output: 42
Program Complete
Output: 346
Program Complete
Output: 8805
Program Complete
Output: 26432
Program Complete
Output: 105738
Program Complete
Output: 42
Program Complete
Output: 346
Program Complete
Output: 8805
Program Complete
Output: 35230
Program Complete
Output: 105707
Program Complete
Output: 42
Program Complete
Output: 346
Program Complete
Output: 1055
Program Complete
Output: 26530
Program Complete
Output: 106130
Program Complete
Output: 42
Program Comp

In [81]:
biggest

298586

In [23]:
og_program[0]=99

99

In [31]:
og_program = OffsetArray([parse(Int64,i) for i in split("3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0",',')],-1)
og_program[1]=4
og_program[5]=0
og_program

17-element OffsetArray(::Array{Int64,1}, 0:16) with eltype Int64 with indices 0:16:
    3
    4
    3
   16
 1002
    0
   10
   16
    1
   16
   15
   15
    4
   15
   99
    0
    0

In [42]:
[4,0][1]

4